# [Exercise 0](https://colab.research.google.com/github/kokchun/Maskininlarning-AI21/blob/main/Exercises/E00_linear_regression.ipynb#scrollTo=XaaBo62vK1Bp)

In [ ]:
import matplotlib.pyplot as pyplot
import numpy
import pandas
import seaborn

## 0

### a)

In [ ]:
numpy.random.seed(42)
x_minutes_per_month = numpy.abs(numpy.random.normal(loc=100, scale=100, size=(400)))
x_minutes_per_month[:5]

In [ ]:
randomized_error = numpy.random.normal(loc=0, scale=50, size=(400))
randomized_error[:5]

In [ ]:
y_sek_per_month = 2 * x_minutes_per_month + 25 + randomized_error

In [ ]:
fig, ax = pyplot.subplots(dpi=100)
seaborn.scatterplot(x=x_minutes_per_month, y=y_sek_per_month)
ax.set(
    title="Simulated 400 datapoints including boundaries",
    xlabel="Call length in minutes",
    ylabel="SEK/month",
)
ax.axvline(300, dashes=(1, 3), color="green", label="maximum")
ax.axhline(0, dashes=(1, 3), color="orange", label="minimum")
ax.legend()

### b)

In [ ]:
# remove over 300 min
# remove costs that are negative
clean_statement = (0 <= y_sek_per_month) & (x_minutes_per_month <= 300)
x_minutes_per_month_cleaned = x_minutes_per_month[clean_statement]
y_sek_per_month_cleaned = y_sek_per_month[clean_statement]
len(x_minutes_per_month_cleaned), len(y_sek_per_month_cleaned), len(x_minutes_per_month), len(y_sek_per_month)

In [ ]:
y_sek_per_month_ground_truth = 2 * x_minutes_per_month + 25

In [ ]:
fig, ax = pyplot.subplots(dpi=100)
seaborn.scatterplot(x=x_minutes_per_month_cleaned, y=y_sek_per_month_cleaned)
seaborn.lineplot(x=x_minutes_per_month, y=y_sek_per_month_ground_truth, color='red', label='Ground truth')
ax.set(
    title="Cleaned outliers",
    xlabel="Call length in minutes",
    ylabel="SEK/month",
)
ax.legend()
pyplot.xlim(0,300)

### c)

In [ ]:
y_sek_per_month_cleaned.shape, x_minutes_per_month_cleaned.shape

In [ ]:
phone_data_month = pandas.DataFrame({'Minutes':x_minutes_per_month_cleaned, 'Cost':y_sek_per_month_cleaned})
phone_data_month.head()

## 1

### a) b)

In [77]:
def train_test_split(
    X: pandas.DataFrame,
    y: pandas.DataFrame,
    train_fraction: float = 0.7,
    random_state: int = 42,
    replace=False,
):
    def sample(df: pandas.DataFrame):
        return df.sample(
            n=int(len(df) * train_fraction), random_state=random_state, replace=False
        )

    def select_not_in_list(
        original: pandas.DataFrame, filtering_data_frame: pandas.DataFrame
    ):
        return original[~original.isin(filtering_data_frame).iloc[:, 0]] # https://www.codegrepper.com/code-examples/python/pandas+dataframe+select+rows+not+in+list

    X_train, y_train = (sample(X), sample(y))

    X_test, y_test = (select_not_in_list(X, X_train), select_not_in_list(y,y_train))

    return {
        "train_data": {"X": X_train, "y": y_train},
        "test_data": {"X": X_test, "y": y_test},
    }


phone_data_month_train_test_split = train_test_split(
    X=pandas.DataFrame(phone_data_month["Minutes"]),
    y=pandas.DataFrame(phone_data_month["Cost"]),
)
train_data = phone_data_month_train_test_split["train_data"]
test_data = phone_data_month_train_test_split["test_data"]


train_data["X"].shape, train_data['y'].shape, test_data["X"].shape, test_data['y'].shape

((266, 1), (266, 1), (115, 1), (115, 1))